# 2 Simulating Price and Funding Fees

1. Compared to the volatility of the underlying, what is the likelihood that I will profit in this scenario? (Portfolio Size > 1)
   1. How volatile should the underlying be to make a profit?
   2. What is the likelihood that a leverage long position will make money?
   3. How is Funding fees affected by movement in price?
2. What is the expected amount that I will pay for cryptocurrencies? - screen through all possible tokens
3. What is the difference between me opening a position on Hyperliquid vs Binance?
   1. Notional
   2. Percentage per hour 
4. What is the optimal platform?
<!-- 7. How can I scale it to other platforms? -->

## Assumptions
- 2 Independent Random Distributions